# Pairwise correlation

In [336]:
from numpy.random import permutation

from sklearn.metrics import pairwise_distances
import numpy as np
X = np.random.rand(1000, 50000)
Y = X**2
zero = np.random.rand(1000, 50000)
test = pairwise_distances(X, Y, ) #.mean() #"correlation").mean()

In [377]:
%%time
v2v(X, Y)

invalid
CPU times: user 2.46 s, sys: 0 ns, total: 2.46 s
Wall time: 2.92 s


1.0

In [386]:
np.array(1).shape

()

In [383]:
%%time
v2v_1d(X, Y).shape

invalid
CPU times: user 1.62 s, sys: 964 µs, total: 1.62 s
Wall time: 1.61 s


(50000,)

In [382]:
%%time
v2v(X, Y)

invalid
CPU times: user 2.57 s, sys: 0 ns, total: 2.57 s
Wall time: 2.83 s


1.0

In [376]:
from numpy.random import permutation

def v2v_old(true, pred, metric="cosine"):
    r = pairwise_distances(true, pred, metric)
    ns = len(r)
    pick = np.random.choice(ns, ns)
    invalid = (pick==np.arange(ns))
    while invalid.any():
        print("invalid")
        pick[invalid] = np.random.choice(invalid.sum())
        invalid = (pick==np.arange)
    correct = np.diag(r) < r[np.arange(ns), pick]
    acc = correct.sum() / len(correct) 
    return acc


def v2v_1d(true, pred, metric="cosine"):
    r = np.abs(true[None] - pred[:, None])
    ns = len(r)
    pick = np.random.choice(ns, ns)
    invalid = (pick==np.arange(ns))
    while invalid.any():
        print("invalid")
        pick[invalid] = np.random.choice(invalid.sum())
        invalid = (pick==np.arange)
    diag = np.arange(ns)
    correct = r[diag, diag] < r[diag, pick] # of shape (1000, 100)
    acc = correct.sum(0) / correct.shape[-1]
    return acc


def v2v(true, pred, metric="cosine"):
    assert len(true) == len(pred)
    ns = len(true)
    first = permutation(ns)
    second = permutation(ns)
    while (first==second).any():
        print("invalid")
        first[first==second] = np.random.choice((first==second).sum())
    
    correct = 0
    for i, j in zip(first, second):
        r = pairwise_distances(true[[i, j]], pred[[i, j]], metric)
        diag = np.diag(r).sum()
        correct += 1*(diag < r.sum() - diag)
    acc = correct / ns
    return acc


def v2v_1d(true, pred):
    assert len(true) == len(pred)
    ns = len(true)
    first = permutation(ns)
    second = permutation(ns)
    while (first==second).any():
        print("invalid")
        first[first==second] = np.random.choice((first==second).sum())
    
    correct = 0
    for i, j in zip(first, second):
        r = np.abs(true[[i, j]][None] - pred[[i, j]][:, None])
        diag = r[0, 0] + r[1, 1]
        correct += 1*(diag < r.sum((0, 1)) - diag)
    acc = correct / ns
    return acc


In [349]:
np.random.permutation(10)

array([4, 3, 8, 6, 5, 7, 9, 1, 2, 0])

In [ ]:
def v2v(true, pred, metric=correlate):
    ns = len(true)
    pick = np.random.choice(ns, ns)
    

# New models ("bert" "T5")



* bert-base-uncased
* squeezebert/squeezebert-mnli
* xlnet-base-cased
* roberta-base
* transfo-xl-wt103
* allenai/longformer-base-4096
* t5-base
* DialoGPT-small

In [29]:
"""* bert-base-uncased
* squeezebert/squeezebert-mnli
* xlnet-base-cased
* roberta-base
* transfo-xl-wt103
* allenai/longformer-base-4096
* t5-base
* DialoGPT-small""".replace("* ", "").split()

['bert-base-uncased',
 'squeezebert/squeezebert-mnli',
 'xlnet-base-cased',
 'roberta-base',
 'transfo-xl-wt103',
 'allenai/longformer-base-4096',
 't5-base',
 'DialoGPT-small']

*distilbert
*bert-base-multilingual-uncased (to compare the effect of learning on other language)
*gpt2-medium
*gpt2-large (?)
*xlnet-base-cased
*roberta-base
*transfo-xl-wt103
*t5-base

*albert-base-v1
*DialoGPT-small
*roberta-base-openai-detector
*distilbert-base-uncased
*distilgpt2
*microsoft/layoutlm-base-uncased
*allenai/longformer-base-4096

In [253]:
import logging

import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

def extract_causal_hidden(model, input_ids):
    assert input_ids.size(0) == 1
    nw = input_ids.size(1)
    input_ids = input_ids.expand((nw, nw))
    mask = 1-torch.triu(torch.ones_like(input_ids))
    outputs = model(input_ids, attention_mask=mask, output_hidden_states=True)
    outputs = outputs.hidden_states
    outputs = [out[torch.arange(nw), torch.arange(nw)][None] for out in outputs]
    print(outputs[0].shape, "extract_causal_hidden")
    return outputs


def yield_hidden_states(model, inputs, max_len=256):
    idx = np.arange(inputs["input_ids"].size(1))
    splits = np.array_split(idx, len(idx) // max_len + 1)
    for idx in splits:
        batch_inputs = {k: v[:, idx] for k, v in inputs.items()}
        outputs = model(**batch_inputs, output_hidden_states=True)

        # FIX
        outputs = list(outputs[2])
        outputs[0] = model.base_model.wte.forward(batch_inputs["input_ids"])
        print("HERE OK ")
        hidden_states = torch.stack(outputs).squeeze(1)
        yield hidden_states

def yield_hidden_states_general(model, inputs, max_len=256, 
                                force_causal=False, fix_embeddings=True):
    inputs = inputs["input_ids"]
    assert inputs.size(0) == 1 
    idx = np.arange(inputs.size(1)) #np.arange(inputs["input_ids"].size(1))
    splits = np.array_split(idx, len(idx) // max_len + 1)
    with torch.no_grad():
        for idx in splits:

            batch_inputs = inputs[:, idx].clone()

            if force_causal:
                nw = len(idx)
                batch_inputs = batch_inputs.expand((nw, nw))
                mask = 1-torch.triu(torch.ones_like(batch_inputs))
                out = model(batch_inputs, output_hidden_states=True, attention_mask=mask)
                out = out.hidden_states
                out = [x[torch.arange(nw), torch.arange(nw)][None] for x in out]

            else:
                out = model(batch_inputs, output_hidden_states=True)
                out = list(out.hidden_states)   

            if fix_embeddings: # TOFIX, not clean, not general
                out[0] = model.base_model.wte.forward(batch_inputs)

            hidden_states = torch.stack(out).squeeze(1)
            yield hidden_states


def map_word_to_inputs(words, tokenizer):
    mapping = {}
    idx = 0
    inputs = tokenizer("", return_tensors="pt", add_special_tokens=False)
    inputs = {k: v.long() for k, v in inputs.items()}
    for i, word in enumerate(words):
        word_inpt = tokenizer(word, return_tensors="pt")
        for k, v in inputs.items():
            inputs[k] = torch.cat([inputs[k], word_inpt[k]], dim=1).long()
        ntok = word_inpt[k].size(1)
        mapping[i] = torch.arange(idx, idx + ntok + 1)
        idx += ntok
    return inputs, mapping


def get_transformer_embeddings(words, model_name="gpt2", agg="mean"):

    # Load
    # words = events.word_raw.values
    assert agg in ["sum", "mean", "last"]

    print(f"Loading model {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    if model_name == "gpt2":
        opts = dict(force_causal=False, fix_embeddings=True)
    else:
        opts = dict(force_causal=True, fix_embeddings=False)
        

    # Inputs
    print("Processed words", words[:5])
    inputs, mapping = map_word_to_inputs(words, tokenizer)
    # import pdb

    # pdb.set_trace()

    # Inference
    with torch.no_grad():
        hidden_states = torch.cat(list(yield_hidden_states_general(model, inputs, **opts)), dim=1)

        if agg == "mean":
            logging.warning("Averaging BPE")
            # Mapping
            word_level_hidden_states = torch.stack(
                [
                    torch.mean(hidden_states[:, mapping[i][0] : mapping[i][-1]], dim=1)
                    for i in range(len(words))
                ],
                dim=1,
            )
        elif agg == "sum":
            logging.warning("Summing BPE")
            # Mapping
            word_level_hidden_states = torch.stack(
                [
                    torch.sum(hidden_states[:, mapping[i][0] : mapping[i][-1]], dim=1)
                    for i in range(len(words))
                ],
                dim=1,
            )

        elif agg == "last":
            logging.warning("Cutting to the last BPE")
            # Mapping
            word_level_hidden_states = torch.stack(
                [hidden_states[:, (mapping[i][-1] - 1)] for i in range(len(words))],
                dim=1,
            )

    assert word_level_hidden_states.size(1) == len(words)
    return word_level_hidden_states.detach()


In [393]:
words = ["I", "am", "a", "cat"]
map_word_to_inputs(words, tokenizer)[0]["input_ids"].shape

torch.Size([1, 12])

In [163]:
model = AutoModel.from_pretrained("gpt2")

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [169]:
out = torch.cat(list(yield_hidden_states(model, input_ids)), dim=1)

HERE OK 


In [395]:
import sklearn
sklearn.__version__

'0.24.1'

In [171]:
out.shape

torch.Size([13, 4, 768])

# Select models

In [239]:
models = ["albert-base-v1",
#"DialoGPT-small",
"roberta-base-openai-detector",
"distilbert-base-uncased",
"distilgpt2",
"microsoft/layoutlm-base-uncased",
"allenai/longformer-base-4096"]


models += ['bert-base-uncased',
 'squeezebert/squeezebert-mnli',
 'xlnet-base-cased',
 'roberta-base',
 'transfo-xl-wt103',
 'allenai/longformer-base-4096',
 #'t5-base',
 #'DialoGPT-small',
]

# Check function

In [243]:
cd /private/home/ccaucheteux/hasson-syntaxe-vs-semantics

/private/home/ccaucheteux/hasson-syntaxe-vs-semantics


In [245]:
ls src

/bin/bash: /public/apps/anaconda3/2020.11/lib/libtinfo.so.6: no version information available (required by /bin/bash)
__init__.py                     get_control_features.py
__pycache__/                    get_features.py
constants.py                    get_features_old.py
encode.py                       get_lexical_phone_error.py
exp_multisubjects.py            manning_proj_embeddings.py
exp_multitasks.py               paths.py
exp_single_task.py              preprocess_stim.py
exp_singlesubject_mutifeats.py  syntactic_equivalences.py
fir.py                          task_dataset.py
fmri_old.py                     transformer_embeddings.py
get_bold.py                     transformer_errors.py
get_concat_features.py          utils/


In [257]:
from src.preprocess_stim import get_stimulus
stim = get_stimulus("pieman", lower=False)
tokens = stim.word_raw.values[:600]

In [258]:
from transformers import AutoModelForCausalLM

causals = []
errors = []
for model_name in models:
    print(model_name)
    with torch.no_grad():
        emb = get_transformer_embeddings(tokens, model_name, agg="sum")
        print(emb.shape)

albert-base-v1
Loading model albert-base-v1
Processed words ['I' 'began' 'my' 'illustrious' 'career']


torch.Size([13, 600, 768])
roberta-base-openai-detector
Loading model roberta-base-openai-detector
Processed words ['I' 'began' 'my' 'illustrious' 'career']


torch.Size([13, 600, 768])
distilbert-base-uncased
Loading model distilbert-base-uncased
Processed words ['I' 'began' 'my' 'illustrious' 'career']


torch.Size([7, 600, 768])
distilgpt2
Loading model distilgpt2


Some weights of GPT2Model were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

torch.Size([13, 600, 768])
roberta-base
Loading model roberta-base
Processed words ['I' 'began' 'my' 'illustrious' 'career']


torch.Size([13, 600, 768])
transfo-xl-wt103
Loading model transfo-xl-wt103


/private/home/ccaucheteux/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


Processed words ['I' 'began' 'my' 'illustrious' 'career']


TypeError: forward() got an unexpected keyword argument 'attention_mask'

# Check causality

In [231]:
from transformers import AutoModelForCausalLM


causals = []
errors = []
for model_name in models:
    print(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    try:
        model = AutoModelForCausalLM.from_pretrained(model_name)
        print("Okay for AutoModelForCausalLM")
    except Exception as e:
        print(e, "causalLM")
        try:
            model = AutoModelForCausalLM.from_pretrained(model_name)
        except Exception as e:
            try:
                model = AutoModel.from_pretrained(model_name)
            except Exception as e:
                print(e, "automodel")
        
    with torch.no_grad():
        input_ids = torch.LongTensor([[10, 11, 12, 20]])
        input_ids = {"input_ids":input_ids}
        
        #out = extract_causal_hidden(model, input_ids)
        #out = out[1][0, 2]
        
        opts = dict(force_causal=True, fix_embeddings=False)
        out = torch.cat(list(yield_hidden_states_general(model, input_ids, **opts)), dim=1)
        print("out.shape", out.shape)
        out = out[1, 2]
        print("out.shape", out.shape)
        
        input_ids = torch.LongTensor([[10, 11, 12, 20, 10, 11, 5, 2, 5]])
        input_ids = {"input_ids":input_ids}
        
        #out2 = extract_causal_hidden(model, input_ids)
        #out2 = out2[1][0, 2]
        
        out2 = torch.cat(list(yield_hidden_states_general(model, input_ids, **opts)), dim=1)
        print("out2.shape", out2.shape)
        out2 = out2[1, 2]
        print("out2.shape", out2.shape)

        print(torch.allclose(out, out2))
        if torch.allclose(out, out2, atol=1e-3, rtol=1e-3):
            causals.append(model_name)
        else:
            errors.append(model_name)

albert-base-v1
Unrecognized configuration class <class 'transformers.models.albert.configuration_albert.AlbertConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of CamembertConfig, XLMRobertaConfig, RobertaConfig, BertConfig, OpenAIGPTConfig, GPT2Config, TransfoXLConfig, XLNetConfig, XLMConfig, CTRLConfig, ReformerConfig, BertGenerationConfig, XLMProphetNetConfig, ProphetNetConfig. causalLM
n_splits 1
[0 1 2 3]
tensor([[10, 11, 12, 20]])
out.shape torch.Size([13, 4, 768])
out.shape torch.Size([768])
n_splits 1
[0 1 2 3 4 5 6 7 8]
tensor([[10, 11, 12, 20, 10, 11,  5,  2,  5]])
out2.shape torch.Size([13, 9, 768])
out2.shape torch.Size([768])
False
roberta-base-openai-detector


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


KeyboardInterrupt: 

In [187]:
opts = dict(force_causal=True, fix_embeddings=False)
opts = dict(force_causal=True, fix_embeddings=False)
out = torch.cat(list(yield_hidden_states_general(model, input_ids, **opts)), dim=1)
out

tensor([[[-0.9056,  0.2749,  2.0964,  ...,  0.6398, -0.0260, -0.1634],
         [-0.5949, -0.0208,  0.4261,  ..., -0.1078,  1.3618,  0.5799],
         [-0.3211,  0.0869, -1.1684,  ...,  0.3524,  0.6197,  0.2317],
         ...,
         [-0.3214,  0.1721,  0.4360,  ...,  0.4868, -0.6630,  1.0812],
         [-0.1438,  0.2584,  1.0188,  ...,  1.3151,  0.5453,  0.1916],
         [-0.0418,  0.5118,  0.9573,  ...,  0.4976, -0.4530,  1.0069]],

        [[-0.8079,  0.4702,  2.0856,  ...,  0.6243,  0.9628, -0.7691],
         [ 0.1957, -0.5334,  0.4716,  ..., -0.0964,  0.9694, -0.1964],
         [-0.1026,  0.2669, -1.3320,  ...,  0.8108,  0.2486,  0.1447],
         ...,
         [ 0.6706,  0.0726,  0.6357,  ...,  0.5045,  0.1864,  1.6206],
         [ 0.6126,  0.1873,  0.8639,  ...,  0.9788,  0.3206,  0.5816],
         [ 0.3499,  0.2055,  0.9875,  ...,  0.6285,  0.6360,  1.1422]],

        [[-0.4876,  0.2747,  1.6616,  ...,  0.3690,  1.1764, -0.5214],
         [ 0.8741, -1.0890,  0.6164,  ...,  0